In [9]:
# Class Scheduling Optimization using DOcplex and Excel Dataset
# Author: Fred, Tahidul, Daniel
# Date: 2025-03-31

import pandas as pd
import random
from docplex.mp.model import Model
from datetime import datetime, timedelta
from itertools import product

# -----------------------------
# 1. Load and Preprocess Data
# -----------------------------

xls = pd.ExcelFile("Data Set - Class Schedule.xlsx")

courses_df = xls.parse('Courses Offered Fall 2025')
professors_df = xls.parse('Professors')
qualification_df = xls.parse('Qualification')
classrooms_df = xls.parse('Classrooms')

courses_df.columns = courses_df.columns.str.strip()
professors_df.columns = professors_df.columns.str.strip()
qualification_df.columns = qualification_df.columns.str.strip()
classrooms_df.columns = classrooms_df.columns.str.strip()

courses_df['Course ID'] = courses_df['Department Code'].astype(str) + ' ' + courses_df['Course Number'].astype(str)
professors_df['Professor ID'] = professors_df['Faculty Name']
classrooms_df['Room ID'] = classrooms_df['Building'].astype(str) + '-' + classrooms_df['Classroom'].astype(str)

courses = courses_df['Course ID'].dropna().unique().tolist()
professors = professors_df['Professor ID'].dropna().unique().tolist()
rooms = classrooms_df['Room ID'].dropna().unique().tolist()
room_capacities = dict(zip(classrooms_df['Room ID'], classrooms_df['Capacity']))
enrollment_dict = dict(zip(courses_df['Course ID'], courses_df['Max Enrollement'].fillna(0).astype(int)))

qualification_matrix = {
    course: {
        prof: int(course in qualification_df.columns and not qualification_df[qualification_df['Faculty Name'] == prof][course].isna().all())
        for prof in professors
    }
    for course in courses
}

preferences = {}
for prof in professors:
    qualified_courses = [c for c in courses if qualification_matrix[c][prof] == 1]
    random.shuffle(qualified_courses)
    preferences[prof] = {course: rank + 1 for rank, course in enumerate(qualified_courses)}

# -----------------------------
# 2. Define Days and Time Slots
# -----------------------------

days = ['M', 'T', 'W', 'R', 'F']
start_time = datetime.strptime("08:00", "%H:%M")
end_time = datetime.strptime("17:00", "%H:%M")
delta = timedelta(minutes=30)

time_of_day = []
current = start_time
while current <= end_time:
    time_of_day.append(current.strftime("%H:%M"))
    current += delta

# Cartesian product of days and time slots
day_time_slots = list(product(days, time_of_day))

# Convert course meeting days to list of days
def parse_days(day_str):
    return day_str.replace('TU', 'T').replace('TH', 'R').replace('/', '').split()

course_days = {
    row['Course ID']: list(day for day in 'MTWRF' if day in row['Day'].replace('TU', 'T').replace('TH', 'R'))
    for _, row in courses_df.iterrows()
}

# -----------------------------
# 3. Build Optimization Model
# -----------------------------

model = Model(name="ClassSchedulingWithDays")

x = model.binary_var_dict(
    [(c, p, r, d, t)
     for c in courses for p in professors for r in rooms for (d, t) in day_time_slots
     if qualification_matrix[c][p] and d in course_days[c]],
    name="x"
)

# Constraint 1: Each course must be assigned on all required days at the same time and room with same professor
for c in courses:
    for p in professors:
        for r in rooms:
            for t in time_of_day:
                occurrences = [x.get((c, p, r, d, t), 0) for d in course_days[c]]
                if all(key in x for key in [(c, p, r, d, t) for d in course_days[c]]):
                    model.add_constraint(model.sum(occurrences) == len(course_days[c]))

# Constraint 2: No professor conflict
for p in professors:
    for d, t in day_time_slots:
        model.add_constraint(model.sum(x.get((c, p, r, d, t), 0) for c in courses for r in rooms) <= 1)

# Constraint 3: No room conflict
for r in rooms:
    for d, t in day_time_slots:
        model.add_constraint(model.sum(x.get((c, p, r, d, t), 0) for c in courses for p in professors) <= 1)

# Constraint 4: Room capacity >= enrollment
for (c, p, r, d, t), var in x.items():
    model.add_constraint(var * enrollment_dict[c] <= room_capacities[r])

# Custom Constraint: Max 2 courses per professor total (not occurrences)
for p in professors:
    model.add_constraint(model.sum(
        model.max([x.get((c, p, r, d, t), 0) for r in rooms for d, t in day_time_slots if (c, p, r, d, t) in x])
        for c in courses) <= 2)

# -----------------------------
# 4. Objective Function
# -----------------------------

model.minimize(model.sum(
    x.get((c, p, r, d, t), 0) * preferences.get(p, {}).get(c, 100)
    for (c, p, r, d, t) in x
))

# -----------------------------
# 5. Solve and Output Results
# -----------------------------

solution = model.solve(log_output=True)

if solution:
    print("\nStatus:", model.solve_details.status)
    print("Objective value (Total Preference Score):", model.objective_value)
    print("\nSchedule:")
    for (c, p, r, d, t), var in x.items():
        if var.solution_value > 0.5:
            print(f"Course {c} assigned to Prof {p} in Room {r} on {d} at {t}")
else:
    print("\nNo feasible solution found.")

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Presolve time = 0.28 sec. (34.16 ticks)

Root node processing (before b&c):
  Real time             =    0.30 sec. (49.51 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.30 sec. (49.51 ticks)

No feasible solution found.


84

760

In [21]:
# Load the Excel file
xls = pd.ExcelFile("Data Set - Class Schedule.xlsx")

# View all sheet names to verify what’s available
print(xls.sheet_names)


['Colleges and Departments', 'Course Catalog', 'Professors', 'Qualification', 'Courses Offered Fall 2025', 'Classrooms']


In [23]:
# Parse relevant sheets
courses_df = xls.parse('Courses Offered Fall 2025')
professors_df = xls.parse('Professors')
qualification_df = xls.parse('Qualification')
classrooms_df = xls.parse('Classrooms')

# Clean column names
for df in [courses_df, professors_df, qualification_df, classrooms_df]:
    df.columns = df.columns.str.strip()


In [31]:
# Create Course ID like "ENTOM 305"
courses_df['Course ID'] = courses_df['Department Code'].astype(str) + ' ' + courses_df['Course Number'].astype(str)

# Use Faculty Name as Professor ID
professors_df['Professor ID'] = professors_df['Faculty Name']

# Create Room ID like "BuildingA-101"
classrooms_df['Room ID'] = classrooms_df['Building'].astype(str) + '-' + classrooms_df['Classroom'].astype(str)


In [37]:
# Enrollment dictionary: course → max enrollment
enrollment_dict = dict(zip(courses_df['Course ID'], courses_df['Max Enrollement'].fillna(0).astype(int)))

# Room capacity dictionary: room → capacity
room_capacities = dict(zip(classrooms_df['Room ID'], classrooms_df['Capacity']))

# Qualification matrix: course → {professor: 0 or 1}
qualification_matrix = {
    course: {
        prof: int(course in qualification_df.columns and not qualification_df[qualification_df['Faculty Name'] == prof][course].isna().all())
        for prof in professors
    }
    for course in courses
}


In [39]:
qualification_matrix

{'ENTOM 300': {'Frank H. Arthur': 0,
  'James F. Campbell': 0,
  'Ming-Shun Chen': 0,
  'Raymond Cloyd': 0,
  'Lee Cohnstaedt': 1,
  'Srinivas Kambhampati': 0,
  'Tania Kim': 1,
  'Berlin Luxelly Londono Renteria': 1,
  'Jeremy L. Marshall': 1,
  'Brian P. McCornack': 0,
  'John P. Michaud': 0,
  'William Morrison III': 0,
  'Dana Nayduch': 1,
  'Cassandra Olds': 0,
  'Weston Opitz': 1,
  'Brenda Oppert': 0,
  'Yoonseong Park': 0,
  'Thomas W. Phillips': 1,
  'Erin Scully': 0,
  'Kristopher Silver': 1,
  'Michael Smith': 1,
  'Brian Spiesman': 1,
  'Robert Jeffery Whitworth': 1,
  'Kun Yan Zhu': 1,
  'Gregory Zolnerowich': 0,
  'Sarah Zukoff': 0,
  'Ludek Zurek': 0},
 'ENTOM 301': {'Frank H. Arthur': 1,
  'James F. Campbell': 1,
  'Ming-Shun Chen': 1,
  'Raymond Cloyd': 1,
  'Lee Cohnstaedt': 1,
  'Srinivas Kambhampati': 0,
  'Tania Kim': 0,
  'Berlin Luxelly Londono Renteria': 0,
  'Jeremy L. Marshall': 1,
  'Brian P. McCornack': 0,
  'John P. Michaud': 0,
  'William Morrison III': 0,

In [43]:
professors = professors_df['Professor ID'].unique().tolist()
professors

['Frank H. Arthur',
 'James F. Campbell',
 'Ming-Shun Chen',
 'Raymond Cloyd',
 'Lee Cohnstaedt',
 'Srinivas Kambhampati',
 'Tania Kim',
 'Berlin Luxelly Londono Renteria',
 'Jeremy L. Marshall',
 'Brian P. McCornack',
 'John P. Michaud',
 'William Morrison III',
 'Dana Nayduch',
 'Cassandra Olds',
 'Weston Opitz',
 'Brenda Oppert',
 'Yoonseong Park',
 'Thomas W. Phillips',
 'Erin Scully',
 'Kristopher Silver',
 'Michael Smith',
 'Brian Spiesman',
 'Robert Jeffery Whitworth',
 'Kun Yan Zhu',
 'Gregory Zolnerowich',
 'Sarah Zukoff',
 'Ludek Zurek']

In [45]:
# Load and clean
courses_df = xls.parse('Courses Offered Fall 2025')
courses_df.columns = courses_df.columns.str.strip()

# Build course IDs
courses_df['Course ID'] = courses_df['Department Code'].astype(str) + ' ' + courses_df['Course Number'].astype(str)

# Extract course list
courses = courses_df['Course ID'].dropna().unique().tolist()

# Extract meeting days per course
def extract_days(day_str):
    return [d for d in 'MTWRF' if d in day_str.replace('TU', 'T').replace('TH', 'R')]

course_days = {
    row['Course ID']: extract_days(row['Day']) for _, row in courses_df.iterrows()
}

# Extract enrollments
enrollment_dict = dict(zip(courses_df['Course ID'], courses_df['Max Enrollement'].fillna(0).astype(int)))


In [49]:
enrollment_dict,course_days

({'ENTOM 300': 50,
  'ENTOM 301': 50,
  'ENTOM 305': 50,
  'ENTOM 306': 50,
  'ENTOM 350': 50,
  'ENTOM 589': 40,
  'ENTOM 602': 40,
  'ENTOM 621': 40,
  'ENTOM 625': 40,
  'ENTOM 635': 40,
  'ENTOM 649': 40,
  'ENTOM 655': 40,
  'ENTOM 657': 40,
  'ENTOM 660': 40,
  'ENTOM 675': 40,
  'ENTOM 680': 40,
  'ENTOM 692': 40,
  'ENTOM 710': 30,
  'ENTOM 732': 30,
  'ENTOM 799': 30,
  'ENTOM 800': 20,
  'ENTOM 805': 20,
  'ENTOM 810': 20,
  'ENTOM 825': 20,
  'ENTOM 830': 20,
  'ENTOM 835': 20,
  'ENTOM 837': 20,
  'ENTOM 840': 20,
  'ENTOM 849': 20,
  'ENTOM 857': 20,
  'ENTOM 860': 20,
  'ENTOM 875': 20,
  'ENTOM 880': 20,
  'ENTOM 885': 20},
 {'ENTOM 300': ['M', 'W', 'F'],
  'ENTOM 301': ['T', 'R'],
  'ENTOM 305': ['T', 'R'],
  'ENTOM 306': ['M', 'W', 'F'],
  'ENTOM 350': ['M', 'W'],
  'ENTOM 589': ['M', 'W', 'F'],
  'ENTOM 602': ['T', 'R'],
  'ENTOM 621': ['T', 'R'],
  'ENTOM 625': ['T', 'R'],
  'ENTOM 635': ['T', 'R'],
  'ENTOM 649': ['M', 'T', 'W', 'R', 'F'],
  'ENTOM 655': ['T', 'R'],